# LIBRERIAS Y DEPENDENCIAS

In [48]:
import pandas as pd # --> PARA MANIPULAR, LIMPIAR Y ANALIZAR DATOS DE DIFERENTES FORMATIOS (csv,parquet, json...)
import numpy as np # --> CALCULO NUMERICO EN MATRICES N-DIMENSIONALES
import re # --> PERMITE BUSCAR Y MANIPULAR CADENAS DE TEXTO MEDIANTE PATRONES REGULARES
import json # MANEJO DE DATOS EN FORMATO JSON


# 1. PREPARACION DE DATOS (FASE ETL)

##  1A. Manejo de datos del dataset movies

In [49]:
# LECTURA DEL DATASET EN CUESTION
df = pd.read_csv('../Datasets/movies_dataset.csv')


C:\Users\USER\AppData\Local\Temp\ipykernel_12644\1585465743.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Datasets/movies_dataset.csv')


In [50]:
df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [51]:
df.shape

(45466, 24)

## - Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0. FUNCION para verificar que cada dato en la columna sea un numero, sino conversion de Nan a 0

In [52]:
# Funcion para rellenar Nulos y '' a 0 en ambas columnas
def clean_column(df, column_name):
    #remplazar '' strings a Nan, si hay
    df[column_name] = df[column_name].replace('', float('nan'))

    #valores no numericos a Nan
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')

    # remplazar NaN o valores nulos a 0
    df[column_name] = df[column_name].fillna(0)

    return df

In [53]:
# for loop para aplicar la funcion de limpieza en las columnas budget 
columns_to_clean = ['budget', 'revenue']
for column in columns_to_clean:
    df = clean_column(df, column)

## - Crear la columna retorno de inversión, llamada return con los campos revenue y budget dividiendo estas dos últimas revenue / budget. En caso que no haya datos disponibles para calcularlo, deberá tomar el valor 0

In [54]:
# Division de revenue / budget para columna return, 0 si denominador es 0
df['return'] = np.where(df['budget'] != 0, df['revenue'] / df['budget'], 0)

In [55]:
# Tipo de variables
columns_to_check = ['return', 'budget', 'revenue']
selected_dtypes = df[columns_to_check].dtypes
print(selected_dtypes)


return     float64
budget     float64
revenue    float64
dtype: object


In [56]:
df.shape

(45466, 25)

## - Los valores nulos del campo release date deben eliminarse

In [57]:
# Eliminacion de nulos a partir de columna release date
df = df.dropna(subset=['release_date'])

In [58]:
# Conteo total de nulos
df['release_date'].isna().sum()

0

In [59]:
# Conversion a datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

tipo_release= df['release_date'].dtype
print(tipo_release)

datetime64[ns]


In [60]:
df.shape

(45379, 25)

## - De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [61]:
# Cambio de formato a año-mes-dia
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

In [62]:
# Verificar si quedo algun nulo
df['release_date'].isna().sum()

3

In [63]:
# Eliminacion de los 3 nulos resultantes
df = df.dropna(subset=['release_date'])
df['release_date'].isna().sum()

0

In [64]:
# Creacion de columna Año de estreno 
df['release_year'] = df['release_date'].dt.year

## - Eliminar las columnas que no serán utilizadas, video, imdb_id, adult, original_title, poster_path y homepage.

In [65]:
# Eliminacion de las columnas indicadas en el enunciado
df=df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage','original_language'])

In [66]:
df.head(7)

,belongs_to_collection,budget,genres,id,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000,1995
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760,1995
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000,1995
5,NaN,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,"Obsessive master thief, Neil McCauley leads a ...",17.924927,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,7.7,1886.0,3.123947,1995
6,NaN,58000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",11860,An ugly duckling having undergone a remarkable...,6.677277,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,6.2,141.0,0.000000,1995


In [67]:
df.shape

(45376, 19)

## - Extraccion de datos de interes anidados en columnas complejas

In [68]:
# ID y info de columnas complejas
target_columns = ['id','belongs_to_collection', 'production_companies', 'genres', 'production_countries', 'spoken_languages']
df[target_columns]

,id,belongs_to_collection,production_companies,genres,production_countries,spoken_languages
0,862,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,NaN,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,15602,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,NaN,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
4,11862,"{'id': 96871, 'name': 'Father of the Bride Col...","[{'name': 'Sandollar Productions', 'id': 5842}...","[{'id': 35, 'name': 'Comedy'}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
...,...,...,...,...,...,...
45460,30840,NaN,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...","[{'iso_639_1': 'en', 'name': 'English'}]"
45462,111109,NaN,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'id': 18, 'name': 'Drama'}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]","[{'iso_639_1': 'tl', 'name': ''}]"
45463,67758,NaN,"[{'name': 'American World Pictures', 'id': 6165}]","[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
45464,227506,NaN,"[{'name': 'Yermoliev', 'id': 88753}]",[],"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",[]


In [69]:
# FUNCION para conversion decimales (floats) a NaN en las columnas de string anidadas
def replace_floats_with_nan(df, columns):
    
    for column in columns:
        df[column] = df[column].apply(lambda x: np.nan if isinstance(x, float) else x)
    return df

In [70]:
# Lista de columnas que se desean procesar
columns_to_process = ['belongs_to_collection', 'genres', 'spoken_languages', 'production_countries']

# Aplicar la función al DataFrame
df = replace_floats_with_nan(df, columns_to_process)


In [71]:
#FUNCION PARA TRABAJAR COLUMNA "belongs_to_collection"

# Función para extraer el valor de 'name' si la fila comienza con '{', de lo contrario devuelve el valor original
def extract_name_if_dict(belongs_to_collection_data):
    if isinstance(belongs_to_collection_data, str) and belongs_to_collection_data.strip().startswith('{'):
        # Usar una expresión regular para buscar 'name': 'NOMBRE'
        match = re.search(r"'name': '([^']*)'", belongs_to_collection_data)
        if match:
            return match.group(1)  # Retorna el nombre encontrado
    return belongs_to_collection_data  # Retorna el valor original si no comienza con '{'

# Aplicar la función a toda la columna 'belongs_to_collection' y crear una nueva columna 'collection_names'
df['collection_names'] = df['belongs_to_collection'].apply(extract_name_if_dict)

# Mostrar las primeras filas para verificar los resultados
print(df[['belongs_to_collection', 'collection_names']])

                                   belongs_to_collection  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...   
1                                                    NaN   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...   
3                                                    NaN   
4      {'id': 96871, 'name': 'Father of the Bride Col...   
...                                                  ...   
45460                                                NaN   
45462                                                NaN   
45463                                                NaN   
45464                                                NaN   
45465                                                NaN   

                     collection_names  
0                Toy Story Collection  
1                                 NaN  
2           Grumpy Old Men Collection  
3                                 NaN  
4      Father of the Bride Collection  
...                               ...  

In [72]:
# FUNCION PARA TRABAJAR LA COLUMNA "genres/genero"

# Función para extraer los valores de 'name' en filas que comienzan con '['
def extract_genres_names(genres_data):
    if isinstance(genres_data, str) and genres_data.strip().startswith('['):
        # Usar una expresión regular para encontrar todas las ocurrencias de 'name': 'NOMBRE'
        names = re.findall(r"'name': '([^']*)'", genres_data)
        return names if names else np.nan
    return np.nan  # Retornar NaN si la fila no comienza con '['

# Aplicar la función a toda la columna 'genres' y crear una nueva columna 'genre_names'
df['genre_names'] = df['genres'].apply(extract_genres_names)

# Mostrar las primeras filas para verificar los resultados
print(df[['genres', 'genre_names']])


                                                  genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2      [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3      [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                         [{'id': 35, 'name': 'Comedy'}]   
...                                                  ...   
45460  [{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...   
45462                      [{'id': 18, 'name': 'Drama'}]   
45463  [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...   
45464                                                 []   
45465                                                 []   

                        genre_names  
0       [Animation, Comedy, Family]  
1      [Adventure, Fantasy, Family]  
2                 [Romance, Comedy]  
3          [Comedy, Drama, Romance]  
4                          [Comedy]  
...                             ...  
45460      [D

In [73]:
# FUNCIÓN PARA CONVERTIR LISTA DE GENEROS A STRINGS CON NORMALIZACIÓN DE ESPACIOS
def list_to_string(x):
    if isinstance(x, list):
        # Eliminar espacios en blanco de cada elemento antes de unirlos
        return ', '.join([genre.strip() for genre in x])
    elif pd.isnull(x):
        return x
    else:
        # Eliminar espacios en blanco si es una cadena simple
        return str(x).strip()

# Aplicar la función a la columna 'genre_names'
df['genre_names'] = df['genre_names'].apply(list_to_string)

# Verificar el resultado
print(df['genre_names'])


0         Animation, Comedy, Family
1        Adventure, Fantasy, Family
2                   Romance, Comedy
3            Comedy, Drama, Romance
4                            Comedy
                    ...            
45460        Drama, Action, Romance
45462                         Drama
45463       Action, Drama, Thriller
45464                           NaN
45465                           NaN
Name: genre_names, Length: 45376, dtype: object


In [74]:
# FUNCION PARA CONVERTIR ELIMINAR VALORES NUMERICOS EN LA COLUMNA GENRES/GENEROS

def is_numeric_only(x):
    return str(x).isdigit()

# Eliminacion de filas donde 'genre_names' es numero
df = df[~df['genre_names'].apply(is_numeric_only)]

print(df['genre_names'])

0         Animation, Comedy, Family
1        Adventure, Fantasy, Family
2                   Romance, Comedy
3            Comedy, Drama, Romance
4                            Comedy
                    ...            
45460        Drama, Action, Romance
45462                         Drama
45463       Action, Drama, Thriller
45464                           NaN
45465                           NaN
Name: genre_names, Length: 45376, dtype: object


In [75]:
# FUNCION PARA TRABAJAR LA COLUMNA DE IDIOMA

# Función para extraer los valores de 'name' en filas que comienzan con '['
def extract_language_names(spoken_languages_data):
    if isinstance(spoken_languages_data, str) and spoken_languages_data.strip().startswith('['):
        # Usar una expresión regular para encontrar todas las ocurrencias de 'name': 'NOMBRE'
        names = re.findall(r"'name': '([^']*)'", spoken_languages_data)
        return names if names else np.nan
    return np.nan  # Retornar NaN si la fila no comienza con '['

# Aplicar la función a toda la columna 'spoken_languages' y crear una nueva columna 'language_names'
df['language_names'] = df['spoken_languages'].apply(extract_language_names)

# Mostrar las primeras filas para verificar los resultados
df[['spoken_languages', 'language_names']]

,spoken_languages,language_names
0,"[{'iso_639_1': 'en', 'name': 'English'}]",[English]
1,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...","[English, Français]"
2,"[{'iso_639_1': 'en', 'name': 'English'}]",[English]
3,"[{'iso_639_1': 'en', 'name': 'English'}]",[English]
4,"[{'iso_639_1': 'en', 'name': 'English'}]",[English]
...,...,...
45460,"[{'iso_639_1': 'en', 'name': 'English'}]",[English]
45462,"[{'iso_639_1': 'tl', 'name': ''}]",[]
45463,"[{'iso_639_1': 'en', 'name': 'English'}]",[English]
45464,[],NaN


In [76]:
# FUNCION PARA ELIMINAR LISTA Y ESPACIOS ENTRE PALABRAS // NORMALIZACION DE IDIOMAS
def list_to_string(x):
    if isinstance(x, list):
        # Eliminacion de espacios entre textos
        result = ', '.join([lang.strip() for lang in x])
    elif pd.isnull(x):
        return x
    else:
        # Eliminacion de espacios para palabras unicas
        result = str(x).strip()

    # Normalizacion de algunos idiomas relevantes
    result = result.replace('日本語', 'Japones')
    result = result.replace('Français', 'Frances')
    result = result.replace('Português', 'Portugues')

    return result

# Uso de la funcion sobre columna 'language_names'
df['language_names'] = df['language_names'].apply(list_to_string)

print(df['language_names'])

0                 English
1        English, Frances
2                 English
3                 English
4                 English
               ...       
45460             English
45462                    
45463             English
45464                 NaN
45465             English
Name: language_names, Length: 45376, dtype: object


In [77]:
#FUNCION PARA TRABAJAR LA COLUMNA "production_contries"

# Función para extraer los valores de 'name' en filas que comienzan con '['
def extract_country_names(production_countries_data):
    if isinstance(production_countries_data, str) and production_countries_data.strip().startswith('['):
        # Usar una expresión regular para encontrar todas las ocurrencias de 'name': 'NOMBRE'
        names = re.findall(r"'name': '([^']*)'", production_countries_data)
        return names if names else np.nan
    return np.nan  # Retornar NaN si la fila no comienza con '['

# Aplicar la función a toda la columna 'production_countries' y crear una nueva columna 'country_names'
df['country_names'] = df['production_countries'].apply(extract_country_names)

# Mostrar las primeras filas para verificar los resultados
print(df[['production_countries', 'country_names']])

                                    production_countries  \
0      [{'iso_3166_1': 'US', 'name': 'United States o...   
1      [{'iso_3166_1': 'US', 'name': 'United States o...   
2      [{'iso_3166_1': 'US', 'name': 'United States o...   
3      [{'iso_3166_1': 'US', 'name': 'United States o...   
4      [{'iso_3166_1': 'US', 'name': 'United States o...   
...                                                  ...   
45460  [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...   
45462      [{'iso_3166_1': 'PH', 'name': 'Philippines'}]   
45463  [{'iso_3166_1': 'US', 'name': 'United States o...   
45464           [{'iso_3166_1': 'RU', 'name': 'Russia'}]   
45465   [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]   

                                           country_names  
0                             [United States of America]  
1                             [United States of America]  
2                             [United States of America]  
3                             [United State

## - Elimnacion de las columnas con informacion anidada

In [78]:
df=df.drop(columns=['belongs_to_collection','production_companies','genres','production_countries','spoken_languages'])

# 1B. Carga de datos del segundo dataset credits.csv que tiene informacion importante (actor, director) en columnas complejas

In [79]:
df_B = pd.read_csv('../Datasets/credits.csv')

In [80]:
df_B

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [81]:
#FUNCION PARA TRABAJAR LA COLUMNA CAST

# Función para extraer todos los nombres de la columna 'cast'
def extract_actor_names(cast_data):
    try:
        # Usar una expresión regular para buscar todos los patrones de 'name': 'NOMBRE'
        names = re.findall(r"'name': '([^']*)'", cast_data)
        return names if names else None
    except TypeError:
        return None  # Manejo de errores si el dato no es una cadena

# Probar la función con un ejemplo específico
row_index = 0  # Reemplaza esto con el índice de la fila que quieres analizar
cast_data = df_B['cast'].iloc[row_index]  # Obtener los datos de la fila
actor_names = extract_actor_names(cast_data)
print(actor_names)

['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney', 'Wallace Shawn', 'John Ratzenberger', 'Annie Potts', 'John Morris', 'Erik von Detten', 'Laurie Metcalf', 'R. Lee Ermey', 'Sarah Freeman', 'Penn Jillette']


In [82]:
# Aplicar la función a toda la columna 'cast' y crear una nueva columna 'actor_names'
df_B['actor_names'] = df_B['cast'].apply(extract_actor_names)

# Mostrar las primeras filas para verificar los resultados
print(df_B[['cast', 'actor_names']])

                                                    cast  \
0      [{'cast_id': 14, 'character': 'Woody (voice)',...   
1      [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2      [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3      [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4      [{'cast_id': 1, 'character': 'George Banks', '...   
...                                                  ...   
45471  [{'cast_id': 0, 'character': '', 'credit_id': ...   
45472  [{'cast_id': 1002, 'character': 'Sister Angela...   
45473  [{'cast_id': 6, 'character': 'Emily Shaw', 'cr...   
45474  [{'cast_id': 2, 'character': '', 'credit_id': ...   
45475                                                 []   

                                             actor_names  
0      [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...  
1      [Robin Williams, Jonathan Hyde, Kirsten Dunst,...  
2      [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...  
3      [Whitney Houston, Angela Bassett, Lo

In [83]:
# CONVERSION DE LA LISTA A STRING DE ACTORES CON NORMALIZACIÓN DE ESPACIOS
def list_to_string(x):
    if isinstance(x, list):
        # Strip spaces from each element before joining
        return ', '.join([actor.strip() for actor in x])
    elif pd.isnull(x):
        return x
    else:
        # Strip spaces if it's a single string
        return str(x).strip()

# Aplicar la función a la columna 'actor_names'
df_B['actor_names'] = df_B['actor_names'].apply(list_to_string)

# Verificar el resultado
print(df_B['actor_names'])

0        Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...
1        Robin Williams, Jonathan Hyde, Kirsten Dunst, ...
2        Walter Matthau, Jack Lemmon, Ann-Margret, Soph...
3        Whitney Houston, Angela Bassett, Loretta Devin...
4        Steve Martin, Diane Keaton, Martin Short, Kimb...
                               ...                        
45471            Leila Hatami, Kourosh Tahami, Elham Korda
45472    Angel Aquino, Perry Dizon, Hazel Orencio, Joel...
45473    Erika Eleniak, Adam Baldwin, Julie du Page, Ja...
45474    Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...
45475                                                 None
Name: actor_names, Length: 45476, dtype: object


In [84]:
#FUNCION PARA TRABAJAR LA COLUMNA CREW

# Función para extraer el nombre del director
def extract_director_name(crew_data):
    try:
        # Usar una expresión regular para buscar 'Director', 'name': 'NOMBRE'
        match = re.search(r"'Director', 'name': '([^']*)'", crew_data)
        if match:
            # Retornar el nombre del director
            return match.group(1)
        return None  # Retornar None si no se encuentra un director
    except TypeError:
        return None  # Manejo de errores si el dato no es una cadena

# Probar la función con un ejemplo específico
row_index = 0  # Reemplaza esto con el índice de la fila que quieres analizar
crew_data = df_B['crew'].iloc[0]  # Obtener los datos de la fila
director_name = extract_director_name(crew_data)
print(director_name)



John Lasseter


In [85]:
# Aplicar la función a toda la columna 'crew' y crear una nueva columna 'director_name'
df_B['director_name'] = df_B['crew'].apply(extract_director_name)

# Mostrar las primeras filas para verificar los resultados
print(df_B[['crew', 'director_name']].head())

                                                crew    director_name
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    John Lasseter
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...     Joe Johnston
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...    Howard Deutch
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  Forest Whitaker
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...    Charles Shyer


In [86]:
df_B.shape

(45476, 5)

## 1C. REDUCCION DE DATAFRAMES Y MERGE CSV FINAL

In [87]:
df_B=df_B.drop(columns=['cast','crew'])

In [88]:
df_B.to_csv('../Datasets/cleaned_crew.csv', index=False)

In [89]:
# EN COLUMNA ID convertir valores que no sean numeros a Nan para UNIR dataframes
df['id'] = pd.to_numeric(df['id'], errors='coerce')
#Eliminar valores Nan 
df = df.dropna(subset=['id'])

In [90]:
# UNIR los dataframe con INNER JOIN por id

df_mix= pd.merge(df,df_B, on='id', how='inner')


In [91]:
# Porcentajes de nulos por columnas en el dataset final combinado
porcentage_nulos = (df_mix.isnull().sum() / len(df_mix)) * 100
print(porcentage_nulos)

budget               0.000000
id                   0.000000
overview             2.070361
popularity           0.000000
release_date         0.000000
revenue              0.000000
runtime              0.541242
status               0.176014
tagline             55.061495
title                0.000000
vote_average         0.000000
vote_count           0.000000
return               0.000000
release_year         0.000000
collection_names    90.105828
genre_names          5.245209
language_names       8.303448
country_names       13.685067
actor_names          5.201206
director_name        2.215573
dtype: float64


## Eliminacion de nulos asociados a la columna de paises 'country_names' y columnas innecesarias para el modelo

In [92]:
df_mix= df_mix.dropna(subset=['country_names'])

In [93]:
df_mix=df_mix.drop(columns=['country_names','collection_names','tagline','id','status','runtime'])

In [94]:
df_mix.to_csv('../Datasets/cleaned_dataset.csv',index=False)